# Imports

In [36]:
#!pip install pygame numpy

In [37]:
import pygame
import numpy as np
import math
import random
import copy
import time
from queue import PriorityQueue
import os


# Constants and Paths

In [38]:
# Constants
SCREEN_WIDTH, SCREEN_HEIGHT = int(2616 / 2.2), int(1816 / 2.2)
CELL_SIZE = 5
GRID_WIDTH, GRID_HEIGHT = SCREEN_WIDTH // CELL_SIZE, SCREEN_HEIGHT // CELL_SIZE
BOT_COLOR = (255, 0, 0)
SECOND_BOT_COLOR = (0, 0, 255)
DYNAMIC_OBSTACLE_COLOR = (128, 0, 128)
DYNAMIC_OBSTACLE_RADIUS = 4
PRIORITY_COLORS = {1: (0, 255, 0), 2: (0, 0, 255), 3: (255, 255, 0)}
MAPPING_SPEED = 10
FPS = 60

In [39]:
# Root directory
ROOT_DIR = os.path.abspath(os.path.join(os.getcwd() , ".."))

# Path to images
ASSETS_DIR = os.path.join(ROOT_DIR, "images", "proj_assets")

# Image paths
IMAGE_PATHS = {
    "env": os.path.join(ASSETS_DIR, "env.png"),
    "agents": [
        os.path.join(ASSETS_DIR, "crabs.png"),
        os.path.join(ASSETS_DIR, "crabs2.png"),
    ],
    "dynamic_obstacles": [
        os.path.join(ASSETS_DIR, "do1.png"),
        os.path.join(ASSETS_DIR, "do2.png"),
    ],
    "targets": [
        os.path.join(ASSETS_DIR, "placeholder1.png"),
        os.path.join(ASSETS_DIR, "placeholder2.png"),
        os.path.join(ASSETS_DIR, "placeholder3.png"),
    ],
}

# Environment Functions

These functions are responsible for creating and managing the simulation environment.

### Static Obstacles and Grid Occupancy


In [40]:
def create_occupancy_grid():
    return np.zeros((GRID_WIDTH, GRID_HEIGHT))

def add_circular_obstacle(occupancy_grid, center, radius):
    cx, cy = center
    for x in range(GRID_WIDTH):
        for y in range(GRID_HEIGHT):
            if (x + 0.5 - cx) ** 2 + (y + 0.5 - cy) ** 2 <= radius ** 2:
                occupancy_grid[x][y] = 1

def add_rectangular_obstacle(occupancy_grid, top_left, width, height):
    start_x, start_y = top_left
    for x in range(start_x, min(start_x + width, GRID_WIDTH)):
        for y in range(start_y, min(start_y + height, GRID_HEIGHT)):
            occupancy_grid[x][y] = 1

def add_obstacles(occupancy_grid):
    """Add predefined circular and rectangular obstacles."""
    # Frame
    add_rectangular_obstacle(occupancy_grid, (0, 1), GRID_WIDTH, 15)
    add_rectangular_obstacle(occupancy_grid, (0, 152), GRID_WIDTH, 15)
    add_rectangular_obstacle(occupancy_grid, (0, 1), 17, GRID_HEIGHT)
    add_rectangular_obstacle(occupancy_grid, (222, 1), 17, GRID_HEIGHT)

    # Dining room (Tables then Barrels)
    add_circular_obstacle(occupancy_grid, (146, 133), 10)
    add_circular_obstacle(occupancy_grid, (74, 133), 10)
    add_circular_obstacle(occupancy_grid, (76, 102), 9)
    add_circular_obstacle(occupancy_grid, (38, 118), 11)
    add_circular_obstacle(occupancy_grid, (187, 118), 12)
    add_circular_obstacle(occupancy_grid, (157, 91), 10)

    add_circular_obstacle(occupancy_grid, (90, 115), 3)
    add_circular_obstacle(occupancy_grid, (162, 112), 3)
    add_circular_obstacle(occupancy_grid, (139, 100), 3)
    add_circular_obstacle(occupancy_grid, (181, 96), 3)
    add_circular_obstacle(occupancy_grid, (210, 123), 3)
    add_circular_obstacle(occupancy_grid, (55, 122), 3)
    add_circular_obstacle(occupancy_grid, (24, 93), 3)
    add_circular_obstacle(occupancy_grid, (24, 137), 3)
    add_circular_obstacle(occupancy_grid, (81, 145), 3)


    add_rectangular_obstacle(occupancy_grid, (101, 70), 28, 18)
    add_circular_obstacle(occupancy_grid, (115, 90), 14)
    
    # Inner walls
    add_rectangular_obstacle(occupancy_grid, (0, 68), 33, 5)
    add_rectangular_obstacle(occupancy_grid, (46, 68), 89, 5)
    add_rectangular_obstacle(occupancy_grid, (153, 68), 38, 5)
    add_rectangular_obstacle(occupancy_grid, (208, 68), 38, 5)

    add_rectangular_obstacle(occupancy_grid, (68, 1), 5, 42)
    add_rectangular_obstacle(occupancy_grid, (68, 61), 5, 10)
    add_rectangular_obstacle(occupancy_grid, (183, 1), 5, 70)

    # Kitchen area
    add_rectangular_obstacle(occupancy_grid, (70, 1), 12, 42)
    add_rectangular_obstacle(occupancy_grid, (104, 1), 23, 26)
    add_rectangular_obstacle(occupancy_grid, (101, 53), 27, 18)
    add_rectangular_obstacle(occupancy_grid, (81, 57), 17, 14)
    add_rectangular_obstacle(occupancy_grid, (177, 36), 8, 20)

    add_circular_obstacle(occupancy_grid, (135, 21), 3)
    add_circular_obstacle(occupancy_grid, (143, 21), 3)

    # Crabs room
    add_rectangular_obstacle(occupancy_grid, (35, 32), 19, 12)
    add_rectangular_obstacle(occupancy_grid, (38, 24), 12, 12)

    add_circular_obstacle(occupancy_grid, (33, 52), 3)
    add_circular_obstacle(occupancy_grid, (53, 53), 3)

    # Bathroom
    add_rectangular_obstacle(occupancy_grid, (185, 21), 14, 30)
    add_rectangular_obstacle(occupancy_grid, (185,55),10,8)

In [41]:
def draw_occupancy_grid(screen, occupancy_grid):
    """
    Draw the occupancy grid on the Pygame screen.
    0: Free space, 1: Obstacle, 2: Dynamic obstacle
    """
    for row in range(len(occupancy_grid)):
        for col in range(len(occupancy_grid[0])):
            cell_color = (255, 255, 255)
            if occupancy_grid[row][col] == 2:
                cell_color = (0, 0, 0)
            elif occupancy_grid[row][col] == 1:
                cell_color = (255, 0, 0)
            pygame.draw.rect(screen, cell_color, pygame.Rect(
                col * CELL_SIZE, row * CELL_SIZE, CELL_SIZE, CELL_SIZE))

### Dynamic Obstacles


In [42]:
# Dynamic Obstacles Movement Pattern
def move_dynamic_obstacles(obstacles, occupancy_grid):
    for obstacle in obstacles:
        x, y, dx, dy = obstacle
        stuck = False

        # Clear previous position in the occupancy grid for the entire radius
        for ox in range(-DYNAMIC_OBSTACLE_RADIUS, DYNAMIC_OBSTACLE_RADIUS + 1):
            for oy in range(-DYNAMIC_OBSTACLE_RADIUS, DYNAMIC_OBSTACLE_RADIUS + 1):
                if 0 <= x + ox < GRID_WIDTH and 0 <= y + oy < GRID_HEIGHT and ox**2 + oy**2 <= DYNAMIC_OBSTACLE_RADIUS**2:
                    occupancy_grid[x + ox][y + oy] = 0

        # Introduce a chance to randomly change direction ( 5% Chance )
        if random.randint(1, 100) <= 5: 
            # Randomly pick a new direction (up, down, left, right)
            dx, dy = random.choice([(1, 0), (-1, 0), (0, 1), (0, -1)])  # Random direction: right, left, down, up

        # Calculate new position
        x_new, y_new = x + dx, y + dy

        # Check if new position is valid and not hitting a static obstacle
        if 0 <= x_new < GRID_WIDTH and 0 <= y_new < GRID_HEIGHT:
            collision = False
            for ox in range(-DYNAMIC_OBSTACLE_RADIUS, DYNAMIC_OBSTACLE_RADIUS + 1):
                for oy in range(-DYNAMIC_OBSTACLE_RADIUS, DYNAMIC_OBSTACLE_RADIUS + 1):
                    if (0 <= x_new + ox < GRID_WIDTH and 0 <= y_new + oy < GRID_HEIGHT 
                        and ox**2 + oy**2 <= DYNAMIC_OBSTACLE_RADIUS**2
                        and occupancy_grid[x_new + ox][y_new + oy] == 1):
                        collision = True
                        break
                if collision:
                    break

            if not collision:
                x, y = x_new, y_new
            else:
                stuck = True

        # Reverse direction if stuck or out of bounds
        if stuck or not (0 <= x_new < GRID_WIDTH) or not (0 <= y_new < GRID_HEIGHT):
            dx, dy = -dy, dx
            x_new, y_new = x + dx, y + dy

            # Check for collisions in the new perpendicular direction
            collision = False
            for ox in range(-DYNAMIC_OBSTACLE_RADIUS, DYNAMIC_OBSTACLE_RADIUS + 1):
                for oy in range(-DYNAMIC_OBSTACLE_RADIUS, DYNAMIC_OBSTACLE_RADIUS + 1):
                    if (0 <= x_new + ox < GRID_WIDTH and 0 <= y_new + oy < GRID_HEIGHT 
                        and ox**2 + oy**2 <= DYNAMIC_OBSTACLE_RADIUS**2
                        and occupancy_grid[x_new + ox][y_new + oy] == 1):
                        collision = True
                        break
                if collision:
                    break

            if not collision:
                x, y = x_new, y_new
            else:
                dx, dy = -dx, -dy  # Reverse direction if still stuck

        # Update occupancy grid for new position
        for ox in range(-DYNAMIC_OBSTACLE_RADIUS, DYNAMIC_OBSTACLE_RADIUS + 1):
            for oy in range(-DYNAMIC_OBSTACLE_RADIUS, DYNAMIC_OBSTACLE_RADIUS + 1):
                if 0 <= x + ox < GRID_WIDTH and 0 <= y + oy < GRID_HEIGHT and ox**2 + oy**2 <= DYNAMIC_OBSTACLE_RADIUS**2:
                    occupancy_grid[x + ox][y + oy] = 1

        # Update the obstacle's position and direction
        obstacle[0], obstacle[1], obstacle[2], obstacle[3] = x, y, dx, dy

        
def draw_dynamic_obstacles(screen, dynamic_obstacles, customers):
    # List of image file names corresponding to customer values (0 or 1)

    size = DYNAMIC_OBSTACLE_RADIUS * CELL_SIZE

    for index, (x, y, _, _) in enumerate(dynamic_obstacles):
        # Choose the image based on the customer value (0 or 1)
        image_file = IMAGE_PATHS["dynamic_obstacles"][customers[index]]
        # Load and scale the selected image
        dynamic_obstacle_image = pygame.image.load(image_file)
        dynamic_obstacle_image = pygame.transform.scale(dynamic_obstacle_image, (size * 2, size * 2))

        # Calculate the center position for the obstacle
        center_x = x * CELL_SIZE + CELL_SIZE // 2
        center_y = y * CELL_SIZE + CELL_SIZE // 2

        # Draw the image centered on the calculated position
        screen.blit(dynamic_obstacle_image, (center_x - dynamic_obstacle_image.get_width() // 2, center_y - dynamic_obstacle_image.get_height() // 2))


## Pygame Intialization 

These functions are responsible for initializing the pygame window.

In [43]:
def initialize_pygame(screen_width, screen_height, title="Search Algorithm Simulation"):
    pygame.init()
    screen = pygame.display.set_mode((screen_width, screen_height))
    pygame.display.set_caption(title)
    return screen

In [44]:
def load_and_scale_image(screen_width, screen_height):
    image = pygame.image.load(IMAGE_PATHS["env"])
    return pygame.transform.scale(image, (screen_width, screen_height))

# Environment Mapping

### Robot Class

Responsible for the robot's movement and sensing.



In [45]:
class Robot:
    def __init__(self, environment, start_position):
        self.env = environment
        self.x, self.y = start_position
        while self.env[self.x, self.y] == 1:
            self.x = random.randint(0, GRID_WIDTH - 1)
            self.y = random.randint(0, GRID_HEIGHT - 1)
        self.orientation = random.uniform(0, 2 * math.pi)
        self.path = [(self.x, self.y)]
    
    def move(self):
        max_attempts = 35  # Max retries to find a valid move
        attempts = 0

        while attempts < max_attempts:
            dx = random.choice([-1, 0, 1]) * MAPPING_SPEED
            dy = random.choice([-1, 0, 1]) * MAPPING_SPEED
            nx, ny = self.x + dx, self.y + dy
            
            if 0 <= nx < GRID_WIDTH and 0 <= ny < GRID_HEIGHT:
                if self.env[nx, ny] == 0:
                    self.x, self.y = nx, ny
                    self.path.append((self.x, self.y))
                    if dx != 0 or dy != 0:
                        self.orientation = math.atan2(dy, dx)
                    return
            attempts += 1
        
        # If no valid move is found, robot remains stationary, but we avoid infinite loop.
        print("Robot could not find a valid move after several attempts.")

                        
    def sense(self):
        max_range = 10  # Range around the robot to sense
        measurements = []
        
        for dx in range(-max_range, max_range + 1):
            for dy in range(-max_range, max_range + 1):
                nx, ny = self.x + dx, self.y + dy
                if 0 <= nx < GRID_WIDTH and 0 <= ny < GRID_HEIGHT:
                    if math.sqrt(dx**2 + dy**2) <= max_range:
                        measurements.append((nx, ny))
                        # Stop further exploration in this direction only if an obstacle is hit
                        if self.env[nx, ny] == 1:
                            continue
                else:
                    # Handles cases where the robot might go out of bounds
                    print(f"Skipping out-of-bounds measurement at ({nx}, {ny})")
                    continue
        
        return measurements




### Applying the Log-Odds Update

Responsible for applying the log-odds update to the occupancy grid.


In [46]:
def inverse_sensor_model(cell, robot_pos, env):
    x_cell, y_cell = cell
    x_robot, y_robot = robot_pos
    if env[x_cell, y_cell] == 1:
        return 0.9
    else:
        return 0.3

def update_occupancy_grid(occupancy_grid, measurements, robot, true_map):
    for cell in measurements:
        prob = inverse_sensor_model(cell, (robot.x, robot.y), true_map)
        lx, ly = cell
        l_prior = occupancy_grid[lx, ly]
        l_sensor = math.log(prob / (1 - prob))

        # Update the occupancy grid using log-odds with proper bounds
        updated_l = l_prior + l_sensor
        occupancy_grid[lx, ly] = max(min(updated_l, 50), -50)


def draw(screen, occupancy_grid, robot):
    screen.fill((255, 255, 255)) 
    
    # Draw the occupancy grid
    for x in range(GRID_WIDTH):
        for y in range(GRID_HEIGHT):
            l = occupancy_grid[x, y]
            p = 1 - 1 / (1 + math.exp(l))
            color = (255 * (1 - p), 255 * (1 - p), 255 * (1 - p))
            rect = pygame.Rect(x * CELL_SIZE, y * CELL_SIZE, CELL_SIZE, CELL_SIZE)
            pygame.draw.rect(screen, color, rect)
    
    # Draw the robot
    pygame.draw.circle(screen, (0, 0, 255), (int(robot.x * CELL_SIZE + CELL_SIZE / 2), int(robot.y * CELL_SIZE + CELL_SIZE / 2)), CELL_SIZE // 2)
    
    # Draw the robot's sensing area
    for pos in robot.path:
        pygame.draw.circle(screen, (0, 255, 0), (int(pos[0] * CELL_SIZE + CELL_SIZE / 2), int(pos[1] * CELL_SIZE + CELL_SIZE / 2)), 2)
    
    # Draw the "enlightened" area the robot sensed
    for measurement in robot.sense():
        mx, my = measurement
        pygame.draw.circle(screen, (255, 255, 0), (int(mx * CELL_SIZE + CELL_SIZE / 2), int(my * CELL_SIZE + CELL_SIZE / 2)), 3)


# Graph Mapping Simulation Function

Responsible for running the mapping simulation.



In [47]:
def run_simulation():
    screen = initialize_pygame(SCREEN_WIDTH, SCREEN_HEIGHT)
    clock = pygame.time.Clock()
    occupancy_grid = create_occupancy_grid()
    true_map = np.copy(occupancy_grid)
    add_obstacles(true_map)
    robot = Robot(true_map, (20, 20))
    running = True
    num_steps = 3000
    
    step = 0
    while running and step < num_steps:
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                running = False
        
        robot.move()
        measurements = robot.sense()
        update_occupancy_grid(occupancy_grid, measurements, robot, true_map)
        draw(screen, occupancy_grid, robot)
        pygame.display.flip()
        clock.tick(FPS)
        step += 1
        
        # Stopping condition: If the robot has explored most of the map
        explored_cells = sum(1 for x in range(GRID_WIDTH) for y in range(GRID_HEIGHT) if occupancy_grid[x, y] != 0)

        # Stop when 95% of the map is explored
        if explored_cells >= 0.95 * GRID_WIDTH * GRID_HEIGHT:  
            running = False

    pygame.quit()
    return occupancy_grid 


# A* Algorithm

Responsible for the A* algorithm.




In [48]:
# ----- A* Algorithm with Dynamic Obstacle Handling -----
def heuristic(a, b):
    return abs(a[0] - b[0]) + abs(a[1] - b[1])

def a_star_search(occupancy_grid, start, goal, dynamic_obstacles, second_bot_position):
    open_list = PriorityQueue()
    open_list.put((0, start))
    came_from = {}
    g_score = {start: 0}
    f_score = {start: heuristic(start, goal)}

    while not open_list.empty():
        _, current = open_list.get()

        if current == goal:
            path = []
            while current in came_from:
                path.append(current)
                current = came_from[current]
            path.reverse()
            return path

        x, y = current
        neighbors = [(x - 1, y), (x + 1, y), (x, y - 1), (x, y + 1)]

        for neighbor in neighbors:
            nx, ny = neighbor
            if 0 <= nx < GRID_WIDTH and 0 <= ny < GRID_HEIGHT and occupancy_grid[nx][ny] == 0:
                # Skip this neighbor if it's occupied by a dynamic obstacle or the other bot
                if any((nx == dx and ny == dy) for dx, dy, _, _ in dynamic_obstacles) or (nx, ny) == second_bot_position:
                    continue

                tentative_g_score = g_score[current] + 1
                
                # Update if this path to neighbor is better
                if neighbor not in g_score or tentative_g_score < g_score[neighbor]:
                    came_from[neighbor] = current
                    g_score[neighbor] = tentative_g_score
                    f_score[neighbor] = tentative_g_score + heuristic(neighbor, goal)
                    
                    # Only add to open list if it hasn't been added before
                    if neighbor not in (item[1] for item in open_list.queue):
                        open_list.put((f_score[neighbor], neighbor))

    return []  # Return an empty path if no path is found




# Dijkstra's Algorithm

Responsible for the Dijkstra's Algorithm.




In [49]:
def dijkstra_search(occupancy_grid, start, goal, dynamic_obstacles, second_bot_position):
    open_list = PriorityQueue()
    open_list.put((0, start))  
    came_from = {}
    g_score = {start: 0} 

    while not open_list.empty():
        current_cost, current = open_list.get()

        if current == goal:
            path = []
            while current in came_from:
                path.append(current)
                current = came_from[current]
            path.reverse()
            return path 

        x, y = current
        neighbors = [(x - 1, y), (x + 1, y), (x, y - 1), (x, y + 1)] 

        for neighbor in neighbors:
            nx, ny = neighbor
            if 0 <= nx < GRID_WIDTH and 0 <= ny < GRID_HEIGHT and occupancy_grid[nx][ny] == 0:
                # Skip if occupied by dynamic obstacles or the second bot
                if any((nx == dx and ny == dy) for dx, dy, _, _ in dynamic_obstacles) or (nx, ny) == second_bot_position:
                    continue

                tentative_g_score = g_score[current] + 1 

                # Update the path if we found a lower cost to reach the neighbor
                if neighbor not in g_score or tentative_g_score < g_score[neighbor]:
                    came_from[neighbor] = current
                    g_score[neighbor] = tentative_g_score
                    open_list.put((tentative_g_score, neighbor))

    return []

# Bot and Target Functions

Responsible for drawing the bots and targets.


In [50]:
# ----- Bot and Target -----

def draw_entity(screen, position, entity_type, index):
    size = DYNAMIC_OBSTACLE_RADIUS * CELL_SIZE

    if entity_type == "bot":
        entity_image = pygame.image.load(IMAGE_PATHS["agents"][index - 1])
        entity_image = pygame.transform.scale(entity_image, (size * 2, size * 2))
    elif entity_type == "target":
        entity_image = pygame.image.load(IMAGE_PATHS["targets"][index - 1])
        entity_image = pygame.transform.scale(entity_image, (size, size))
    else:
        return

    center_x = position[0] * CELL_SIZE + CELL_SIZE // 2
    center_y = position[1] * CELL_SIZE + CELL_SIZE // 2
    top_left_x = center_x - entity_image.get_width() // 2
    top_left_y = center_y - entity_image.get_height() // 2

    screen.blit(entity_image, (top_left_x, top_left_y))


def prioritize_targets(targets):
    # Sort targets by priority
    targets.sort(key=lambda t: t["priority"])
    
    bot1_targets = []
    bot2_targets = []

    # Initialize distance trackers for balancing
    bot1_distance = 0
    bot2_distance = 0

    # Iterate over sorted targets to assign them to the bots
    for target in targets:
        if bot1_distance <= bot2_distance:
            bot1_targets.append(copy.deepcopy(target))
            bot1_distance += target["priority"]
            print(f"Assigned target with {target["priority"]} to Bot 1" )
        else:
            bot2_targets.append(copy.deepcopy(target))
            bot2_distance += target["priority"]
            print(f"Assigned target with {target["priority"]} to Bot 2" )

    return bot1_targets, bot2_targets



# Main Simulation Loop

All helper functions for the main loop




In [51]:
# ----- Main Simulation Helper Functions -----
def initialize_environment(screen_width, screen_height, occupancy_grid):
    screen = initialize_pygame(screen_width, screen_height)
    environment_image = load_and_scale_image(screen_width, screen_height)
    
    blend_surface = pygame.Surface((screen_width, screen_height))
    blend_surface.fill((128, 128, 128))  # Fill with 50% gray

    # Update the blend surface based on the occupancy grid
    for x in range(GRID_WIDTH):
        for y in range(GRID_HEIGHT):
            if occupancy_grid[x][y] == 0:  # Free space
                pygame.draw.rect(blend_surface, (255, 255, 255), 
                                 (x * CELL_SIZE, y * CELL_SIZE, CELL_SIZE, CELL_SIZE))
                
    return screen, environment_image, blend_surface

def is_position_free(position, occupancy_grid, dynamic_obstacles):
    x, y = position
    return (occupancy_grid[x][y] == 0 and 
            not any(math.sqrt((x - dx) ** 2 + (y - dy) ** 2) <= DYNAMIC_OBSTACLE_RADIUS 
                    for dx, dy, _, _ in dynamic_obstacles))

def handle_bot_movement(bot_data, occupancy_grid, dynamic_obstacles, navigation_algo):
    for bot in bot_data:
        # Check if the bot has completed all targets
        if bot.get("completed"):
            continue  # Skip processing for bots that have finished their targets
        
        # Check if the bot has a path to follow
        if bot["path_index"] < len(bot["full_path"]):
            next_position = bot["full_path"][bot["path_index"]]

            # Check if the next position is free of obstacles
            if is_position_free(next_position, occupancy_grid, dynamic_obstacles):
                bot["position"] = next_position
                bot["path_index"] += 1

                # Check if the bot has reached the final target position
                if bot["path_index"] == len(bot["full_path"]):
                    print(f"Bot {bot['bot']} has reached its target at {next_position}")

        else:
            # If the bot has completed its path, check if there are more targets to navigate to
            if bot["targets"]:
                next_target = bot["targets"].pop(0)["position"]
                bot["full_path"] = navigation_algo(
                    occupancy_grid, bot["position"], next_target, dynamic_obstacles, bot["position"]
                )
                bot["path_index"] = 0  # Reset path index
                print(f"Bot {bot['bot']} is moving to next target at {next_target}")
            else:
                # If no more targets are available, mark the bot as completed
                bot["completed"] = True
                print(f"Bot {bot['bot']} has completed all assigned targets.")


def render_bots(screen, bot_data, targets):
    for bot in bot_data:
        draw_entity(screen, bot["position"], "bot", bot["bot"])

    for target in targets:
        draw_entity(screen, target["position"], "target", target["priority"])

    pygame.display.flip()



In [52]:
# ----- Main Simulation Loop -----

def main(occupancy_grid, navigation_algo, text_case, target_num = 2):
    # Initialize Pygame screen and environment
    screen, environment_image, blend_surface = initialize_environment(SCREEN_WIDTH, SCREEN_HEIGHT, occupancy_grid)

    # Define dynamic obstacles and customers
    dynamic_obstacles = text_case()
    customers = [random.randint(0, 1) for _ in dynamic_obstacles]
    
    # Define target positions with priorities
    targets = [
        {"position": (205, 135), "priority": 1},
        {"position": (170, 100), "priority": 2},
        {"position": (135, 110), "priority": 3},
        {"position": (60, 50), "priority": 1},
        {"position": (100, 30), "priority": 2}
    ]

    # Split targets between bots
    targets1, targets2 = prioritize_targets(targets[:target_num])

    # Initialize bot data
    bot_data = [
        {"position": (20, 20), "bot": 1, "full_path": [], "path_index": 0, "targets": targets1, "completed": False},
        {"position": (30, 30), "bot": 2, "full_path": [], "path_index": 0, "targets": targets2, "completed": False}
    ]

    # Initialize the timer
    start_time = time.time()
    clock = pygame.time.Clock()
    running = True

    while running:
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                running = False

        # Blit the environment and blend layers
        screen.blit(environment_image, (0, 0))
        screen.blit(blend_surface, (0, 0), special_flags=pygame.BLEND_MULT)

        # Update and render dynamic obstacles
        move_dynamic_obstacles(dynamic_obstacles, occupancy_grid)
        draw_dynamic_obstacles(screen, dynamic_obstacles, customers)

        # Update bot positions
        handle_bot_movement(bot_data, occupancy_grid, dynamic_obstacles, navigation_algo)

        # Render bots and targets
        render_bots(screen, bot_data, targets[:target_num])

        # Check if all bots have completed their targets
        if all(bot.get("completed") for bot in bot_data):
            end_time = time.time()
            elapsed_time = end_time - start_time
            print(f"All bots have completed their assigned targets. Ending simulation.")
            print(f"Time elapsed: {elapsed_time:.2f} seconds.")
            break  # End the simulation loop

        clock.tick(FPS)

    pygame.quit()


Begin the mapping of the enviornment

In [53]:
occupancy_grid = run_simulation()

In [54]:
def invert_occupancy_grid(grid):
    visited = set()
    for x in range(GRID_WIDTH):
        for y in range(GRID_HEIGHT):
            if (x, y) not in visited:
                visited.add((x, y))
                if grid[x][y] < 0:
                    grid[x][y] = 0
                else:
                    grid[x][y] = 1
    return grid

Make a copy of envionments to run two different algorithms on

In [55]:
inverted_occupancy_grid = invert_occupancy_grid(occupancy_grid)
inverted_occupancy_grid_2= np.copy(inverted_occupancy_grid)
inverted_occupancy_grid_3= np.copy(inverted_occupancy_grid)
inverted_occupancy_grid_4= np.copy(inverted_occupancy_grid)
inverted_occupancy_grid_5= np.copy(inverted_occupancy_grid)
inverted_occupancy_grid_6= np.copy(inverted_occupancy_grid)

inverted_occupancy_grid_7= np.copy(inverted_occupancy_grid)
inverted_occupancy_grid_8= np.copy(inverted_occupancy_grid)
inverted_occupancy_grid_9= np.copy(inverted_occupancy_grid)
inverted_occupancy_grid_10= np.copy(inverted_occupancy_grid)

# Testing Scenarios

The normal, the high-traffic, the emergency




In [56]:
def normal():
    dynamic_obstacles = [[30, 35, -1, 0], [25, 25, -1, 0], [80, 60, 1, 0],[100, 80, 1, 0]]
    return dynamic_obstacles

def high_traffic():
    dynamic_obstacles = [[30, 35, -1, 0], [25, 25, -1, 0], [80, 60, 1, 0],[100, 80, 1, 0],
                         [20, 130, 1, 0], [170, 80, 1, 0], [150, 80, 1, 0], [100, 130, 1, 0]]
    return dynamic_obstacles

def emergency():
    dynamic_obstacles = [[30, 35, -1, 0], [25, 25, -1, 0], [80, 60, 1, 0],[100, 80, 1, 0],
                         [20, 130, 1, 0], [170, 80, 1, 0], [150, 80, 1, 0], [100, 130, 1, 0],
                         [180, 75, 1, 0], [175, 85, 1, 0], [90, 80, 1, 0], [105, 135, 1, 0]]
    return dynamic_obstacles

# Run A* Search
Normal Case: 4 Dynamic Obstacles.

In [57]:
main(inverted_occupancy_grid, a_star_search, normal)

Assigned target with 1 to Bot 1
Assigned target with 2 to Bot 2
Bot 1 is moving to next target at (205, 135)
Bot 2 is moving to next target at (170, 100)
Bot 2 has reached its target at (170, 100)
Bot 2 has completed all assigned targets.
Bot 1 has reached its target at (205, 135)
Bot 1 has completed all assigned targets.
All bots have completed their assigned targets. Ending simulation.
Time elapsed: 12.13 seconds.


High traffic case: 8 Dynamic Obstacles.


In [58]:
main(inverted_occupancy_grid_2, a_star_search, high_traffic)

Assigned target with 1 to Bot 1
Assigned target with 2 to Bot 2
Bot 1 is moving to next target at (205, 135)
Bot 2 is moving to next target at (170, 100)
Bot 2 has reached its target at (170, 100)
Bot 2 has completed all assigned targets.
Bot 1 has reached its target at (205, 135)
Bot 1 has completed all assigned targets.
All bots have completed their assigned targets. Ending simulation.
Time elapsed: 14.05 seconds.


Emergency Case: 12 Dynamic Obstacles.

In [59]:
main(inverted_occupancy_grid_3, a_star_search, emergency)

Assigned target with 1 to Bot 1
Assigned target with 2 to Bot 2
Bot 1 is moving to next target at (205, 135)
Bot 2 is moving to next target at (170, 100)
Bot 2 has reached its target at (170, 100)
Bot 2 has completed all assigned targets.
Bot 1 has reached its target at (205, 135)
Bot 1 has completed all assigned targets.
All bots have completed their assigned targets. Ending simulation.
Time elapsed: 14.75 seconds.


# Run Dijkstra's Algorithm
Normal Case: 4 Dynamic Obstacles.

In [60]:
main(inverted_occupancy_grid_4, dijkstra_search, normal)

Assigned target with 1 to Bot 1
Assigned target with 2 to Bot 2
Bot 1 is moving to next target at (205, 135)
Bot 2 is moving to next target at (170, 100)
Bot 2 has reached its target at (170, 100)
Bot 2 has completed all assigned targets.
Bot 1 has reached its target at (205, 135)
Bot 1 has completed all assigned targets.
All bots have completed their assigned targets. Ending simulation.
Time elapsed: 11.20 seconds.


High traffic case: 8 Dynamic Obstacles.


In [61]:
main(inverted_occupancy_grid_5, dijkstra_search, high_traffic)

Assigned target with 1 to Bot 1
Assigned target with 2 to Bot 2
Bot 1 is moving to next target at (205, 135)
Bot 2 is moving to next target at (170, 100)
Bot 2 has reached its target at (170, 100)
Bot 2 has completed all assigned targets.
Bot 1 has reached its target at (205, 135)
Bot 1 has completed all assigned targets.
All bots have completed their assigned targets. Ending simulation.
Time elapsed: 13.08 seconds.


Emergency Case: 12 Dynamic Obstacles.

In [62]:
main(inverted_occupancy_grid_6, dijkstra_search, emergency)

Assigned target with 1 to Bot 1
Assigned target with 2 to Bot 2
Bot 1 is moving to next target at (205, 135)
Bot 2 is moving to next target at (170, 100)
Bot 2 has reached its target at (170, 100)
Bot 2 has completed all assigned targets.
Bot 1 has reached its target at (205, 135)
Bot 1 has completed all assigned targets.
All bots have completed their assigned targets. Ending simulation.
Time elapsed: 14.16 seconds.


# Different Number of Targets
A* Normal Case: 4 Dynamic Obstacles.
Targets = 3

In [63]:
main(inverted_occupancy_grid_7, a_star_search, normal, 3)

Assigned target with 1 to Bot 1
Assigned target with 2 to Bot 2
Assigned target with 3 to Bot 1
Bot 1 is moving to next target at (205, 135)
Bot 2 is moving to next target at (170, 100)
Bot 2 has reached its target at (170, 100)
Bot 2 has completed all assigned targets.
Bot 1 has reached its target at (205, 135)
Bot 1 is moving to next target at (135, 110)
Bot 1 has reached its target at (135, 110)
Bot 1 has completed all assigned targets.
All bots have completed their assigned targets. Ending simulation.
Time elapsed: 15.01 seconds.


A* Normal Case: 4 Dynamic Obstacles.
Targets = 5

In [64]:
main(inverted_occupancy_grid_8, a_star_search, normal, 5)

Assigned target with 1 to Bot 1
Assigned target with 1 to Bot 2
Assigned target with 2 to Bot 1
Assigned target with 2 to Bot 2
Assigned target with 3 to Bot 1
Bot 1 is moving to next target at (205, 135)
Bot 2 is moving to next target at (60, 50)
Bot 2 has reached its target at (60, 50)
Bot 2 is moving to next target at (100, 30)
Bot 2 has reached its target at (100, 30)
Bot 2 has completed all assigned targets.
Bot 1 has reached its target at (205, 135)
Bot 1 is moving to next target at (170, 100)
Bot 1 has reached its target at (170, 100)
Bot 1 is moving to next target at (135, 110)
Bot 1 has reached its target at (135, 110)
Bot 1 has completed all assigned targets.
All bots have completed their assigned targets. Ending simulation.
Time elapsed: 16.44 seconds.


Dijkstra Normal Case: 4 Dynamic Obstacles.
Targets = 3

In [65]:
main(inverted_occupancy_grid_9, dijkstra_search, normal, 3)

Assigned target with 1 to Bot 1
Assigned target with 2 to Bot 2
Assigned target with 3 to Bot 1
Bot 1 is moving to next target at (205, 135)
Bot 2 is moving to next target at (170, 100)
Bot 2 has reached its target at (170, 100)
Bot 2 has completed all assigned targets.
Bot 1 has reached its target at (205, 135)
Bot 1 is moving to next target at (135, 110)
Bot 1 has reached its target at (135, 110)
Bot 1 has completed all assigned targets.
All bots have completed their assigned targets. Ending simulation.
Time elapsed: 14.98 seconds.


Dijkstra Normal Case: 4 Dynamic Obstacles.
Targets = 5

In [66]:
main(inverted_occupancy_grid_10, dijkstra_search, normal, 5)

Assigned target with 1 to Bot 1
Assigned target with 1 to Bot 2
Assigned target with 2 to Bot 1
Assigned target with 2 to Bot 2
Assigned target with 3 to Bot 1
Bot 1 is moving to next target at (205, 135)
Bot 2 is moving to next target at (60, 50)
Bot 2 has reached its target at (60, 50)
Bot 2 is moving to next target at (100, 30)
Bot 2 has reached its target at (100, 30)
Bot 2 has completed all assigned targets.
Bot 1 has reached its target at (205, 135)
Bot 1 is moving to next target at (170, 100)
Bot 1 has reached its target at (170, 100)
Bot 1 is moving to next target at (135, 110)
Bot 1 has reached its target at (135, 110)
Bot 1 has completed all assigned targets.
All bots have completed their assigned targets. Ending simulation.
Time elapsed: 16.34 seconds.
